In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);

=====> /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1), "T" => d[:total_tools], "N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);


make: `/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6' is up to date.


Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean          SD        Naive SE       MCSE         ESS    
         lp__ -39.612289300  3.21503515 0.05083416910 0.1286977701  624.064396
accept_stat__   0.943197167  0.07922159 0.00125260338 0.0019554508 1000.000000
   stepsize__   0.048296850  0.00703668 0.00011125968 0.0011266297   39.009752
  treedepth__   5.953250000  0.50510219 0.00798636681 0.0300925655  281.734524
 n_leapfrog__  78.088000000 36.91011847 0.58360021530 2.1696456995  289.409804
  divergent__   0.000000000  0.00000000 0.00000000000 0.0000000000         NaN
     energy__  46.103521900  4.12900938 0.06528537067 0.1491521758  766.359515
        alpha   1.128738441  0.75244083 0.01189713414 0.0207060104 1000.000000
  a_society.1  -0.215986350  0.24384726 0.00385556374 0.0071578380 1000.000000
  a_society.2   0.035700505  0.21960160 0.00347220623 0.0053465841 1000.000000
  a_society.3  

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*